# Инициализация и получение данных

Ноутбук демонстрирует пример запроса и работы с данными [MyTracker](https://tracker.my.com).

> **Внимание!** Чтобы всё заработало, в ячейку ниже вам необходио вписать UserId и SecretKey! Получить их можно в личном кабинете, если у вас есть права на экспорт сырых данных. Пожалуйста, проконсультируйтесь с [документацией](https://docs.tracker.my.com/api/export-api/access/), как сделать это.

In [12]:
API_USER_ID = 12345
API_SECRET_KEY = '******************'

In [2]:
%pip install mytracker_export_api

Note: you may need to restart the kernel to use updated packages.


In [3]:
from mytracker_export_api import MyTracker
params = {
    'dateFrom': '2024-05-17',
    'dateTo': '2024-05-17',
    'selectors': 'eventName,idAppVersionTitle,idOsTitle,tsEvent,idDevice',
    'idApp': 61475,
    'event': 'customEvents'
}
client = MyTracker(api_user_id=API_USER_ID, api_secret_key=API_SECRET_KEY)
df = client.get_raw_data(params)

In [4]:
df.head()

,eventName,idAppVersionTitle,idOsTitle,tsEvent,idDevice
0,press,2.17.1-f9da73e6,Android,1715961024,52344631831
1,close,2.30.0,Android,1715927918,81114376137
2,close,2.30.0,Android,1715927951,81114376137
3,close,2.30.0,Android,1715927990,81114376137
4,close,2.30.0,Android,1715928030,81114376137


# Постобработка
Конвертация дат, фильтрация

## Преобразование timestampt в дату и время

In [5]:
import pandas as pd
df['eventDateTime'] = pd.to_datetime(df['tsEvent'], unit='s')

In [6]:
df

,eventName,idAppVersionTitle,idOsTitle,tsEvent,idDevice,eventDateTime
0,press,2.17.1-f9da73e6,Android,1715961024,52344631831,2024-05-17 15:50:24
1,close,2.30.0,Android,1715927918,81114376137,2024-05-17 06:38:38
2,close,2.30.0,Android,1715927951,81114376137,2024-05-17 06:39:11
3,close,2.30.0,Android,1715927990,81114376137,2024-05-17 06:39:50
4,close,2.30.0,Android,1715928030,81114376137,2024-05-17 06:40:30
...,...,...,...,...,...,...
6279,swipe,2.17.0-87e8eb36,Android,1715938811,81793521913,2024-05-17 09:40:11
6280,swipe,2.17.0-87e8eb36,Android,1715939478,81793521913,2024-05-17 09:51:18
6281,swipe,2.17.0-87e8eb36,Android,1715939479,81793521913,2024-05-17 09:51:19
6282,swipe,2.17.0-87e8eb36,Android,1715939730,81793521913,2024-05-17 09:55:30


## Фильтрация

### По имени события

In [7]:
df['press' == df['eventName']]

,eventName,idAppVersionTitle,idOsTitle,tsEvent,idDevice,eventDateTime
0,press,2.17.1-f9da73e6,Android,1715961024,52344631831,2024-05-17 15:50:24
60,press,2.30.0,Android,1715927758,81114376137,2024-05-17 06:35:58
61,press,2.30.0,Android,1715927914,81114376137,2024-05-17 06:38:34
62,press,2.30.0,Android,1715927918,81114376137,2024-05-17 06:38:38
63,press,2.30.0,Android,1715927951,81114376137,2024-05-17 06:39:11
...,...,...,...,...,...,...
6266,press,2.17.0-87e8eb36,Android,1715938815,81793521913,2024-05-17 09:40:15
6267,press,2.17.0-87e8eb36,Android,1715938821,81793521913,2024-05-17 09:40:21
6268,press,2.17.0-87e8eb36,Android,1715938828,81793521913,2024-05-17 09:40:28
6269,press,2.17.0-87e8eb36,Android,1715939479,81793521913,2024-05-17 09:51:19


### По номеру девайса

In [8]:
df[81114376137 == df['idDevice']]

,eventName,idAppVersionTitle,idOsTitle,tsEvent,idDevice,eventDateTime
1,close,2.30.0,Android,1715927918,81114376137,2024-05-17 06:38:38
2,close,2.30.0,Android,1715927951,81114376137,2024-05-17 06:39:11
3,close,2.30.0,Android,1715927990,81114376137,2024-05-17 06:39:50
4,close,2.30.0,Android,1715928030,81114376137,2024-05-17 06:40:30
5,close,2.30.0,Android,1715928096,81114376137,2024-05-17 06:41:36
...,...,...,...,...,...,...
447,mt_login,2.30.0,Android,1715927993,81114376137,2024-05-17 06:39:53
448,mt_login,2.30.0,Android,1715928038,81114376137,2024-05-17 06:40:38
449,mt_login,2.30.0,Android,1715928460,81114376137,2024-05-17 06:47:40
450,mt_login,2.30.0,Android,1715929583,81114376137,2024-05-17 07:06:23


### По девайсу и событию вместе

In [9]:
df[(df['eventName'] == 'press') & (81114376137 == df['idDevice'])]

,eventName,idAppVersionTitle,idOsTitle,tsEvent,idDevice,eventDateTime
60,press,2.30.0,Android,1715927758,81114376137,2024-05-17 06:35:58
61,press,2.30.0,Android,1715927914,81114376137,2024-05-17 06:38:34
62,press,2.30.0,Android,1715927918,81114376137,2024-05-17 06:38:38
63,press,2.30.0,Android,1715927951,81114376137,2024-05-17 06:39:11
64,press,2.30.0,Android,1715928096,81114376137,2024-05-17 06:41:36
65,press,2.30.0,Android,1715928222,81114376137,2024-05-17 06:43:42
66,press,2.30.0,Android,1715928226,81114376137,2024-05-17 06:43:46
67,press,2.30.0,Android,1715928372,81114376137,2024-05-17 06:46:12
68,press,2.30.0,Android,1715928374,81114376137,2024-05-17 06:46:14
69,press,2.30.0,Android,1715928376,81114376137,2024-05-17 06:46:16
